In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

def model_forest_biomass(X, t, params):
    """Model with base parameters instead of combined parameters"""
    Y = np.zeros(7)
    
    X = np.maximum(X, np.ones_like(X) * 1e-10)
    
    # Base parameters from table
    s = params[0]      # 0.8
    alpha = params[1]  # 0.05
    beta1 = params[2]  # 0.003
    beta2 = params[3]  # 0.0004
    rho2 = params[4]   # 0.02
    r = params[5]      # 0.9
    v1 = params[6]     # 0.002
    v2 = params[7]     # 0.0001
    theta = params[8]  # 0.5
    lambda_val = params[9]  # 0.05
    sigma = params[10] # 0.001
    phi = params[11]   # 0.1
    phi0 = params[12]  # 0.5
    phi1 = params[13]  # 0.01
    gamma = params[14] # 0.004
    pi_val = params[15] # 0.002
    phi2 = params[16]  # 0.2
    gamma1 = params[17] # 0.01
    rho = params[18]   # 0.01
    rho1 = params[19]  # 0.03
    omega = params[20] # 0.05
    omega1 = params[21] # 0.1
    
    # State variables
    B = X[0]  # Forest Biomass
    W = X[1]  # Wildlife
    N = X[2]  # Human Population
    P = X[3]  # Population Pressure
    H = X[4]  # Human Activities
    T = X[5]  # Technological Efforts
    E = X[6]  # Economic Measures

    # Differential equations using base parameters
    try:
        Y[0] = -s*B - alpha*B*W - beta1*B*N - beta2*(B**2)*H + rho2*B*T
        Y[1] = r*W*(1 - W/B) - v1*W*N - v2*W*H
        Y[2] = theta*N*(1 - N) + lambda_val*beta1*B*N - sigma*N*W
        Y[3] = phi*N - phi0*P - phi1*P*E
        Y[4] = gamma*N + pi_val*beta2*(B**2)*H + phi2*P - gamma1*H
        Y[5] = rho*(1 - B) - rho1*T
        Y[6] = omega*P - omega1*E
    except:
        Y = np.zeros(7)
    
    # Prevent numerical instabilities
    Y = np.clip(Y, -1e3, 1e3)
    return Y

def generate_gaussian_params(n_samples=100):
    """Generate parameters using Gaussian distribution based on table values"""
    np.random.seed(42)
    
    # Parameter values from table
    param_means = [
        0.8,    # s
        0.05,   # alpha
        0.003,  # beta1
        0.0004, # beta2
        0.02,   # rho2
        0.9,    # r
        0.002,  # v1
        0.0001, # v2
        0.5,    # theta
        0.05,   # lambda
        0.001,  # sigma
        0.1,    # phi
        0.5,    # phi0
        0.01,   # phi1
        0.004,  # gamma
        0.002,  # pi
        0.2,    # phi2
        0.01,   # gamma1
        0.01,   # rho
        0.03,   # rho1
        0.05,   # omega
        0.1     # omega1
    ]
    
    param_stds = [mean * 0.1 for mean in param_means]
    
    params_data = np.zeros((n_samples, len(param_means)))
    for i in range(len(param_means)):
        params_data[:, i] = np.maximum(param_means[i] * 0.1, 
                                     np.random.normal(param_means[i], param_stds[i], n_samples))
    
    param_names = [
        's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
        'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
        'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
    ]
    
    return pd.DataFrame(params_data, columns=param_names)

# Generate synthetic data
t = np.linspace(0, 100, 1000)
X0 = [1.0, 0.5, 0.3, 0.2, 0.1, 0.1, 0.1]

synthetic_params = generate_gaussian_params(n_samples=1)
initial_params = synthetic_params.iloc[0].values

try:
    original_solution = odeint(model_forest_biomass, X0, t, args=(tuple(initial_params),))
except:
    print("Error in generating original solution")
    raise

# Add noise to create observed data
np.random.seed(42)
noise_level = 0.02  
observed_data = original_solution + noise_level * np.random.randn(*original_solution.shape)

# Ensure observed data is positive
observed_data = np.maximum(observed_data, 0)

def objective_function(params, t, observed_data):
    """Objective function for least squares optimization"""
    try:
        solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
        residuals = (solution - observed_data) / (np.abs(observed_data) + 1e-6)
        return residuals.ravel()
    except:
        return np.ones(len(t) * 7) * 1e6

# Define bounds based on table values
lower_bounds = [param * 0.5 for param in initial_params]  # 50% of initial value
upper_bounds = [param * 1.5 for param in initial_params]  # 150% of initial value

# Perform least squares fitting with more robust settings
result = least_squares(objective_function, initial_params, 
                      args=(t, observed_data),
                      bounds=(lower_bounds, upper_bounds),
                      method='trf',
                      ftol=1e-8,
                      xtol=1e-8,
                      gtol=1e-8,
                      max_nfev=1000)

fitted_params = result.x

try:
    fitted_solution = odeint(model_forest_biomass, X0, t, args=(tuple(fitted_params),))
except:
    print("Error in generating fitted solution")
    raise

# Plot results
variables = [
    'Forest Biomass', 'Wildlife', 'Human Population', 
    'Population Pressure', 'Human Activities',
    'Technological Efforts', 'Economic Measures'
]

plt.figure(figsize=(20, 15))
for i in range(7):
    plt.subplot(3, 3, i+1)
    plt.plot(t, original_solution[:, i], 'r-', label='Original')
    plt.plot(t, observed_data[:, i], 'b.', label='Observed', alpha=0.3)
    plt.plot(t, fitted_solution[:, i], 'g--', label='Fitted')
    plt.xlabel('Time')
    plt.ylabel(variables[i])
    plt.title(f'{variables[i]}')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

# Print parameter comparison
param_names = [
    's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
    'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
    'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
]

print("\nParameter Comparison:")
print("Parameter | Initial | Fitted")
print("-" * 50)
for name, init, fit in zip(param_names, initial_params, fitted_params):
    print(f"{name:8} | {init:8.6f} | {fit:8.6f}")

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

def model_forest_biomass(X, t, params):
    """Model with base parameters instead of combined parameters"""
    Y = np.zeros(7)
    
    X = np.maximum(X, np.ones_like(X) * 1e-10)
    
    # Base parameters from table A.2
    s = params[0]      # Natural decay rate
    alpha = params[1]  # Impact of wildlife
    beta1 = params[2]  # Human population impact
    beta2 = params[3]  # Human activities impact
    rho2 = params[4]   # Technology impact on biomass
    r = params[5]      # Wildlife growth rate
    v1 = params[6]     # Human impact on wildlife
    v2 = params[7]     # Activity impact on wildlife
    theta = params[8]  # Human population growth
    lambda_val = params[9]  # Resource availability
    sigma = params[10] # Wildlife impact on humans
    phi = params[11]   # Population pressure growth
    phi0 = params[12]  # Natural decay of pressure
    phi1 = params[13]  # Economic impact on pressure
    gamma = params[14] # Human activities growth
    pi_val = params[15] # Resource conversion rate
    phi2 = params[16]  # Pressure impact on activities
    gamma1 = params[17] # Activities decay rate
    rho = params[18]   # Technology growth rate
    rho1 = params[19]  # Technology decay rate
    omega = params[20] # Economic growth rate
    omega1 = params[21] # Economic decay rate
    
    # State variables
    B = X[0]  # Forest Biomass
    W = X[1]  # Wildlife
    N = X[2]  # Human Population
    P = X[3]  # Population Pressure
    H = X[4]  # Human Activities
    T = X[5]  # Technological Efforts
    E = X[6]  # Economic Measures

    # Differential equations using base parameters
    try:
        Y[0] = -s*B - alpha*B*W - beta1*B*N - beta2*(B**2)*H + rho2*B*T
        Y[1] = r*W*(1 - W/B) - v1*W*N - v2*W*H
        Y[2] = theta*N*(1 - N) + lambda_val*beta1*B*N - sigma*N*W
        Y[3] = phi*N - phi0*P - phi1*P*E
        Y[4] = gamma*N + pi_val*beta2*(B**2)*H + phi2*P - gamma1*H
        Y[5] = rho*(1 - B) - rho1*T
        Y[6] = omega*P - omega1*E
    except:
        Y = np.zeros(7)
    
    # Prevent numerical instabilities
    Y = np.clip(Y, -1e3, 1e3)
    return Y

def generate_uniform_params(n_samples=100):
    """Generate parameters using uniform distribution based on table A.2 values"""
    np.random.seed(42)
    
    param_ranges = [
        (0.01, 1.0),      # s      - Natural decay rate
        (0.0001, 0.09),   # alpha  - Impact of wildlife
        (0.0001, 0.3),    # beta1  - Human population impact
        (0.0005, 0.04),   # beta2  - Human activities impact
        (0.0001, 0.6),    # rho2   - Technology impact on biomass
        (0.8045, 1.0),    # r      - Wildlife growth rate (using eta)
        (0.0002, 0.2),    # v1     - Human impact on wildlife
        (0.00002, 0.1),   # v2     - Activity impact on wildlife
        (0.01, 1.0),      # theta  - Human population growth
        (0.0, 1.0),       # lambda - Resource availability
        (0.001, 0.1),     # sigma  - Wildlife impact on humans
        (0.01, 0.9),      # phi    - Population pressure growth
        (0.01, 0.9),      # phi0   - Natural decay of pressure
        (0.0001, 0.5),    # phi1   - Economic impact on pressure
        (0.0001, 0.4),    # gamma  - Human activities growth
        (0.0, 1.0),       # pi     - Resource conversion rate
        (0.02, 0.9),      # phi2   - Pressure impact on activities
        (0.001, 0.05),    # gamma1 - Activities decay rate
        (0.01, 0.8),      # rho    - Technology growth rate
        (0.001, 0.5),     # rho1   - Technology decay rate
        (0.01, 0.5),      # omega  - Economic growth rate
        (0.003, 0.8)      # omega1 - Economic decay rate
    ]
    
    param_means = [
        0.69,    # s
        0.0487,  # alpha
        0.0029,  # beta1
        0.0004,  # beta2
        0.0215,  # rho2
        0.8045,  # r (using eta)
        0.002,   # v1
        0.0001,  # v2
        0.75,    # theta
        0.045,   # lambda
        0.0012,  # sigma
        0.09,    # phi
        0.4323,  # phi0
        0.011,   # phi1
        0.0039,  # gamma
        0.023,   # pi
        0.18,    # phi2
        0.0429,  # gamma1
        0.028,   # rho
        0.0738,  # rho1
        0.0524,  # omega
        0.1067   # omega1
    ]
    
    params_data = np.zeros((n_samples, len(param_ranges)))
    for i, ((min_val, max_val), mean_val) in enumerate(zip(param_ranges, param_means)):
        params_data[:, i] = np.random.uniform(min_val, max_val, n_samples)
    
    param_names = [
        's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
        'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
        'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
    ]
    
    return pd.DataFrame(params_data, columns=param_names), param_ranges, param_means

def objective_function(params, t, observed_data):
    """Objective function for least squares optimization"""
    try:
        solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
        residuals = (solution - observed_data) / (np.abs(observed_data) + 1e-6)
        return residuals.ravel()
    except:
        return np.ones(len(t) * 7) * 1e6

# Set up simulation parameters
t = np.linspace(0, 100, 1000)
X0 = [1.0, 0.5, 0.3, 0.2, 0.1, 0.1, 0.1] #ini value B,W,N,P,H,T,E

# Generate parameters and create synthetic data
uniform_params, param_ranges, param_means = generate_uniform_params(n_samples=1)
initial_params = uniform_params.iloc[0].values

try:
    original_solution = odeint(model_forest_biomass, X0, t, args=(tuple(initial_params),))
except:
    print("Error in generating original solution")
    raise

# Add noise to create observed data
np.random.seed(42)
noise_level = 0.02
observed_data = original_solution + noise_level * np.random.randn(*original_solution.shape)
observed_data = np.maximum(observed_data, 0)

# Define bounds for optimization
lower_bounds = [range[0] for range in param_ranges]
upper_bounds = [range[1] for range in param_ranges]

# Perform least squares fitting
result = least_squares(objective_function, 
                      initial_params, 
                      args=(t, observed_data),
                      bounds=(lower_bounds, upper_bounds),
                      method='trf',
                      ftol=1e-8,
                      xtol=1e-8,
                      gtol=1e-8,
                      max_nfev=1000)

fitted_params = result.x

# Generate fitted solution
try:
    fitted_solution = odeint(model_forest_biomass, X0, t, args=(tuple(fitted_params),))
except:
    print("Error in generating fitted solution")
    raise

# Plot results
variables = [
    'Forest Biomass', 'Wildlife', 'Human Population', 
    'Population Pressure', 'Human Activities',
    'Technological Efforts', 'Economic Measures'
]

plt.figure(figsize=(20, 15))
for i in range(7):
    plt.subplot(3, 3, i+1)
    plt.plot(t, original_solution[:, i], 'r-', label='Original')
    plt.plot(t, observed_data[:, i], 'b.', label='Observed', alpha=0.3)
    plt.plot(t, fitted_solution[:, i], 'g--', label='Fitted')
    plt.xlabel('Time')
    plt.ylabel(variables[i])
    plt.title(f'{variables[i]}')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

# Print parameter comparison
param_names = [
    's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
    'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
    'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
]

print("\nParameter Comparison:")
print("Parameter | Initial | Fitted | Range")
print("-" * 60)
for name, init, fit, (min_val, max_val) in zip(param_names, initial_params, fitted_params, param_ranges):
    print(f"{name:8} | {init:8.6f} | {fit:8.6f} | [{min_val:8.6f}, {max_val:8.6f}]")

In [11]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import os
import json

def model_forest_biomass(X, t, params):
    """Forest biomass conservation model with 7 state variables"""
    Y = np.zeros(7)
    X = np.maximum(X, np.ones_like(X) * 1e-10)
    
    # Parameters
    s, alpha, beta1, beta2, rho2, r, v1, v2, theta = params[:9]
    lambda_val, sigma, phi, phi0, phi1, gamma = params[9:15]
    pi_val, phi2, gamma1, rho, rho1, omega, omega1 = params[15:]
    
    # State variables
    B, W, N, P, H, T, E = X
    
    # Differential equations
    try:
        Y[0] = -s*B - alpha*B*W - beta1*B*N - beta2*(B**2)*H + rho2*B*T  # Forest Biomass
        Y[1] = r*W*(1 - W/B) - v1*W*N - v2*W*H  # Wildlife
        Y[2] = theta*N*(1 - N) + lambda_val*beta1*B*N - sigma*N*W  # Human Population
        Y[3] = phi*N - phi0*P - phi1*P*E  # Population Pressure
        Y[4] = gamma*N + pi_val*beta2*(B**2)*H + phi2*P - gamma1*H  # Human Activities
        Y[5] = rho*(1 - B) - rho1*T  # Technological Efforts
        Y[6] = omega*P - omega1*E  # Economic Measures
    except:
        Y = np.zeros(7)
    
    return np.clip(Y, -1e3, 1e3)

def generate_gaussian_params(n_samples=100):
    """Generate parameters using Gaussian distribution"""
    np.random.seed(42)
    
    param_means = [
        0.8,    # s
        0.05,   # alpha
        0.003,  # beta1
        0.0004, # beta2
        0.02,   # rho2
        0.9,    # r
        0.002,  # v1
        0.0001, # v2
        0.5,    # theta
        0.05,   # lambda
        0.001,  # sigma
        0.1,    # phi
        0.5,    # phi0
        0.01,   # phi1
        0.004,  # gamma
        0.002,  # pi
        0.2,    # phi2
        0.01,   # gamma1
        0.01,   # rho
        0.03,   # rho1
        0.05,   # omega
        0.1     # omega1
    ]
    
    param_stds = [mean * 0.1 for mean in param_means]
    params_data = np.zeros((n_samples, len(param_means)))
    
    for i in range(len(param_means)):
        params_data[:, i] = np.maximum(param_means[i] * 0.1, 
                                     np.random.normal(param_means[i], param_stds[i], n_samples))
    
    param_names = [
        's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
        'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
        'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
    ]
    
    return pd.DataFrame(params_data, columns=param_names)

def generate_uniform_params(n_samples=100):
    """Generate parameters using uniform distribution"""
    np.random.seed(42)
    
    param_ranges = [
        (0.01, 1.0),      # s
        (0.0001, 0.09),   # alpha
        (0.0001, 0.3),    # beta1
        (0.0005, 0.04),   # beta2
        (0.0001, 0.6),    # rho2
        (0.8045, 1.0),    # r
        (0.0002, 0.2),    # v1
        (0.00002, 0.1),   # v2
        (0.01, 1.0),      # theta
        (0.0, 1.0),       # lambda
        (0.001, 0.1),     # sigma
        (0.01, 0.9),      # phi
        (0.01, 0.9),      # phi0
        (0.0001, 0.5),    # phi1
        (0.0001, 0.4),    # gamma
        (0.0, 1.0),       # pi
        (0.02, 0.9),      # phi2
        (0.001, 0.05),    # gamma1
        (0.01, 0.8),      # rho
        (0.001, 0.5),     # rho1
        (0.01, 0.5),      # omega
        (0.003, 0.8)      # omega1
    ]
    
    param_means = [
        0.69, 0.0487, 0.0029, 0.0004, 0.0215, 0.8045, 0.002, 0.0001,
        0.75, 0.045, 0.0012, 0.09, 0.4323, 0.011, 0.0039, 0.023,
        0.18, 0.0429, 0.028, 0.0738, 0.0524, 0.1067
    ]
    
    params_data = np.zeros((n_samples, len(param_ranges)))
    for i, ((min_val, max_val), mean_val) in enumerate(zip(param_ranges, param_means)):
        params_data[:, i] = np.random.uniform(min_val, max_val, n_samples)
    
    param_names = [
        's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
        'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
        'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
    ]
    
    return pd.DataFrame(params_data, columns=param_names), param_ranges, param_means

def objective_function(params, t, observed_data):
    """Objective function for least squares optimization"""
    try:
        solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
        residuals = (solution - observed_data) / (np.abs(observed_data) + 1e-6)
        return residuals.ravel()
    except:
        return np.ones(len(t) * 7) * 1e6

def analyze_simulation_results(results, t, save_path):
    """Analyze simulation results and generate comparison metrics"""
    variables = ['Forest Biomass', 'Wildlife', 'Human Population', 
                'Population Pressure', 'Human Activities',
                'Technological Efforts', 'Economic Measures']
    
    analysis_results = {}
    
    for method in results:
        # Time pattern analysis
        time_patterns = {
            'max_values': np.max(results[method]['original'], axis=0),
            'min_values': np.min(results[method]['original'], axis=0),
            'mean_values': np.mean(results[method]['original'], axis=0),
            'final_values': results[method]['original'][-1],
            'stability_metric': np.std(results[method]['original'], axis=0)
        }
        
        # Calculate convergence rates
        convergence_rates = []
        for i in range(7):
            cut_point = int(0.8 * len(t))
            time_slice = t[cut_point:]
            value_slice = results[method]['original'][cut_point:, i]
            if len(time_slice) > 1:
                polyfit = np.polyfit(time_slice, value_slice, 1)
                convergence_rates.append(polyfit[0])
            else:
                convergence_rates.append(np.nan)
        
        analysis_results[method] = {
            'time_patterns': time_patterns,
            'convergence_rates': convergence_rates
        }
    
    # Create comparison plots
    plt.figure(figsize=(20, 15))
    for i in range(7):
        plt.subplot(3, 3, i+1)
        for method in results:
            plt.plot(t, results[method]['original'][:, i], 
                    label=f'{method} Original', alpha=0.7)
            plt.plot(t, results[method]['fitted'][:, i], 
                    linestyle='--', label=f'{method} Fitted', alpha=0.7)
        
        plt.xlabel('Time')
        plt.ylabel(variables[i])
        plt.title(f'{variables[i]} Comparison')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"{save_path}/time_pattern_comparison.png")
    plt.close()
    
    # Generate summary report
    summary_data = []
    for var_idx, var_name in enumerate(variables):
        for method in results:
            summary_data.append({
                'Variable': var_name,
                'Method': method,
                'Final_Value': analysis_results[method]['time_patterns']['final_values'][var_idx],
                'Mean_Value': analysis_results[method]['time_patterns']['mean_values'][var_idx],
                'Stability': analysis_results[method]['time_patterns']['stability_metric'][var_idx],
                'Convergence_Rate': analysis_results[method]['convergence_rates'][var_idx]
            })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(f"{save_path}/simulation_summary.csv", index=False)
    
    return analysis_results, summary_df

def generate_plot_data(results, analysis_results, summary_df):
    """Generate JSON-formatted data for plotting"""
    t = np.linspace(0, 100, 1000)
    
    # Time series data
    time_data = []
    for i in range(len(t)):
        data_point = {'time': float(t[i])}
        for method in results:
            for j, var in enumerate(['Biomass', 'Wildlife', 'Population']):
                key = f"{method.lower()}{var}"
                data_point[key] = float(results[method]['original'][i, j])
        time_data.append(data_point)
    
    # Final values comparison
    variables = ['Forest Biomass', 'Wildlife', 'Human Population', 
                'Population Pressure', 'Human Activities',
                'Technological Efforts', 'Economic Measures']
    
    final_values = []
    for var in variables:
        var_data = {
            'variable': var,
            'gaussian': float(summary_df[
                (summary_df['Variable'] == var) & 
                (summary_df['Method'] == 'Gaussian')
            ]['Final_Value'].iloc[0]),
            'uniform': float(summary_df[
                (summary_df['Variable'] == var) & 
                (summary_df['Method'] == 'Uniform')
            ]['Final_Value'].iloc[0])
        }
        final_values.append(var_data)
    
    # Stability metrics
    stability_data = []
    for var in variables:
        stability_metrics = {
            'variable': var,
            'gaussian': float(summary_df[
                (summary_df['Variable'] == var) & 
                (summary_df['Method'] == 'Gaussian')
            ]['Stability'].iloc[0]),
            'uniform': float(summary_df[
                (summary_df['Variable'] == var) & 
                (summary_df['Method'] == 'Uniform')
            ]['Stability'].iloc[0])
        }
        stability_data.append(stability_metrics)
    
    return {
        'timeSeriesData': time_data,
        'finalValues': final_values,
        'stabilityMetrics': stability_data
    }

def run_full_analysis(n_samples=100, save_path="results"):
    """Run complete analysis workflow"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Simulation parameters
    t = np.linspace(0, 100, 1000)
    X0 = [1.0, 0.5, 0.3, 0.2, 0.1, 0.1, 0.1]
    
    # Generate parameters
    gaussian_params = generate_gaussian_params(n_samples=1)
    uniform_params, param_ranges, param_means = generate_uniform_params(n_samples=1)
    
    # Save parameter distributions
    gaussian_params.to_csv(f"{save_path}/gaussian_parameters.csv", index=False)
    uniform_params.to_csv(f"{save_path}/uniform_parameters.csv", index=False)
    
    results = {}
    for method, params, ranges in [
        ("Gaussian", gaussian_params.iloc[0].values, None),
        ("Uniform", uniform_params.iloc[0].values, param_ranges)
    ]:
        try:
            # Generate original solution
            original_solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
            
            # Add noise and create observed data
            np.random.seed(42)
            noise_level = 0.02
            observed_data = original_solution + noise_level * np.random.randn(*original_solution.shape)
            observed_data = np.maximum(observed_data, 0)
            
            # Define optimization bounds
            if method == "Gaussian":
                lower_bounds = [param * 0.5 for param in params]
                upper_bounds = [param * 1.5 for param in params]
            else:
                lower_bounds = [range[0] for range in ranges]
                upper_bounds = [range[1] for range in ranges]
            
            # Perform parameter fitting
            result = least_squares(
                objective_function, 
                params,
                args=(t, observed_data),
                bounds=(lower_bounds, upper_bounds),
                method='trf',
                ftol=1e-8,
                xtol=1e-8,
                gtol=1e-8,
                max_nfev=1000
            )
            
            fitted_params = result.x
            fitted_solution = odeint(model_forest_biomass, X0, t, args=(tuple(fitted_params),))
            
            results[method] = {
                'original': original_solution,
                'observed': observed_data,
                'fitted': fitted_solution,
                'initial_params': params,
                'fitted_params': fitted_params
            }
            
            # Save solutions to CSV
            df_original = pd.DataFrame(
                original_solution,
                columns=['Forest_Biomass', 'Wildlife', 'Human_Population', 
                        'Population_Pressure', 'Human_Activities',
                        'Technological_Efforts', 'Economic_Measures']
            )
            df_original['time'] = t
            df_original.to_csv(f"{save_path}/{method.lower()}_original_solution.csv", index=False)
            
            df_fitted = pd.DataFrame(
                fitted_solution,
                columns=['Forest_Biomass', 'Wildlife', 'Human_Population', 
                        'Population_Pressure', 'Human_Activities',
                        'Technological_Efforts', 'Economic_Measures']
            )
            df_fitted['time'] = t
            df_fitted.to_csv(f"{save_path}/{method.lower()}_fitted_solution.csv", index=False)
            
        except Exception as e:
            print(f"Error in {method} method: {str(e)}")
            continue
    
    # Analyze results
    analysis_results, summary_df = analyze_simulation_results(results, t, save_path)
    
    # Generate plot data
    plot_data = generate_plot_data(results, analysis_results, summary_df)
    with open(f"{save_path}/plot_data.json", 'w') as f:
        json.dump(plot_data, f)
    
    # Create additional comparison visualizations
    create_comparison_plots(results, analysis_results, t, save_path)
    
    return results, analysis_results, summary_df, plot_data

def create_comparison_plots(results, analysis_results, t, save_path):
    """Create additional comparative visualizations"""
    variables = ['Forest Biomass', 'Wildlife', 'Human Population', 
                'Population Pressure', 'Human Activities',
                'Technological Efforts', 'Economic Measures']
    
    # 1. Phase Space Plot (Forest Biomass vs Wildlife)
    plt.figure(figsize=(10, 8))
    for method in results:
        plt.plot(results[method]['original'][:, 0], 
                results[method]['original'][:, 1],
                label=f'{method}',
                alpha=0.7)
    plt.xlabel('Forest Biomass')
    plt.ylabel('Wildlife')
    plt.title('Phase Space: Forest Biomass vs Wildlife')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{save_path}/phase_space_plot.png")
    plt.close()
    
    # 2. Parameter Sensitivity Analysis
    plt.figure(figsize=(15, 10))
    param_names = [
        's', 'alpha', 'beta1', 'beta2', 'rho2', 'r', 'v1', 'v2', 
        'theta', 'lambda', 'sigma', 'phi', 'phi0', 'phi1', 'gamma',
        'pi', 'phi2', 'gamma1', 'rho', 'rho1', 'omega', 'omega1'
    ]
    
    for method in results:
        param_changes = np.abs(results[method]['fitted_params'] - 
                             results[method]['initial_params']) / results[method]['initial_params']
        plt.plot(range(len(param_names)), param_changes, 'o-', label=method)
    
    plt.xticks(range(len(param_names)), param_names, rotation=45)
    plt.ylabel('Relative Parameter Change')
    plt.title('Parameter Sensitivity Analysis')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{save_path}/parameter_sensitivity.png")
    plt.close()
    
    # 3. Stability Analysis
    plt.figure(figsize=(12, 8))
    for method in results:
        stability_metrics = analysis_results[method]['time_patterns']['stability_metric']
        plt.bar(np.arange(len(variables)) + (0.35 if method == 'Gaussian' else 0),
                stability_metrics,
                width=0.35,
                label=method,
                alpha=0.7)
    
    plt.xticks(range(len(variables)), variables, rotation=45)
    plt.ylabel('Stability Metric (Standard Deviation)')
    plt.title('Stability Analysis by Variable')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{save_path}/stability_analysis.png")
    plt.close()
    
    # 4. Convergence Analysis
    plt.figure(figsize=(12, 8))
    for method in results:
        convergence_rates = analysis_results[method]['convergence_rates']
        plt.bar(np.arange(len(variables)) + (0.35 if method == 'Gaussian' else 0),
                convergence_rates,
                width=0.35,
                label=method,
                alpha=0.7)
    
    plt.xticks(range(len(variables)), variables, rotation=45)
    plt.ylabel('Convergence Rate')
    plt.title('Convergence Analysis by Variable')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{save_path}/convergence_analysis.png")
    plt.close()

def main():
    """Main execution function"""
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Create results directory
    save_path = "forest_model_results"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Run analysis
    results, analysis_results, summary_df, plot_data = run_full_analysis(save_path=save_path)
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print("-" * 50)
    
    print("\nFinal Values Comparison:")
    final_values = summary_df.pivot(index='Variable', columns='Method', values='Final_Value')
    print(final_values)
    
    print("\nStability Comparison:")
    stability = summary_df.pivot(index='Variable', columns='Method', values='Stability')
    print(stability)
    
    print("\nConvergence Rates:")
    convergence = summary_df.pivot(index='Variable', columns='Method', values='Convergence_Rate')
    print(convergence)
    
    # Save summary to text file
    with open(f"{save_path}/analysis_summary.txt", 'w') as f:
        f.write("Forest Conservation Model Analysis Summary\n")
        f.write("=" * 50 + "\n\n")
        
        f.write("Final Values Comparison:\n")
        f.write(str(final_values) + "\n\n")
        
        f.write("Stability Comparison:\n")
        f.write(str(stability) + "\n\n")
        
        f.write("Convergence Rates:\n")
        f.write(str(convergence) + "\n")

if __name__ == "__main__":
    main()


Summary Statistics:
--------------------------------------------------

Final Values Comparison:
Method                     Gaussian       Uniform
Variable                                         
Economic Measures      1.090089e-01  2.364453e+00
Forest Biomass        -6.006787e-09 -1.525306e-09
Human Activities       2.293570e+00  1.127505e+01
Human Population       1.000000e+00  1.000000e+00
Population Pressure    1.858606e-01  8.713959e-01
Technological Efforts  3.328451e-01  2.400412e+00
Wildlife              -5.826854e-08 -4.113275e-09

Stability Comparison:
Method                 Gaussian   Uniform
Variable                                 
Economic Measures      0.005599  0.494147
Forest Biomass         0.078376  0.099030
Human Activities       0.646886  3.159836
Human Population       0.100836  0.079752
Population Pressure    0.013858  0.086800
Technological Efforts  0.066687  0.486199
Wildlife               0.075413  0.092916

Convergence Rates:
Method                     Gaus